# Using `ibis` to connect to MIMIC IV on Google BigQuery

Environments in Google Colaboratory are not persistent. If we use any software that is not part of teh Google Python Colaboratory environment, we must install it during each session.

We are going to be using Ibis, so this must be installed.

In [ ]:
!pip install ibis-framework[bigquery]

In [1]:
import ibis
from google_auth_oauthlib import flow
import os

### Authenticate using `google.colab`

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

## This is copied straight from the Google documentation.

If you haven't verified the app with Google you get some security warnings you need to click through.

In [3]:
# TODO: Uncomment the line below to set the `launch_browser` variable.
launch_browser = True
#
# The `launch_browser` boolean variable indicates if a local server is used
# as the callback URL in the auth flow. A value of `True` is recommended,
# but a local server does not work if accessing the application remotely,
# such as over SSH or from a remote Jupyter notebook.

appflow = flow.InstalledAppFlow.from_client_secrets_file(
    secret, scopes=["https://www.googleapis.com/auth/bigquery"]
)

if launch_browser:
    appflow.run_local_server()
else:
    appflow.run_console()

credentials = appflow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=351306700747-5t1ni8s29h7mug06vls19q52pbma4f8e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=kJe2M31F9LoWG1JZVokDk4RMjuXHXh&access_type=offline


### Pass the credentials to the `ibis` `connect` function.

In [5]:
conn = ibis.bigquery.connect(
    project_id="mimic4-bq",
    dataset_id='physionet-data.mimic_core')

### Once we connect we can list all the databases we have access to

In [6]:
conn.list_databases()

['eicu_crd_demo',
 'mimic_core',
 'mimic_derived',
 'mimic_hosp',
 'mimic_icu',
 'mimiciii_clinical',
 'mimiciii_demo',
 'mimiciii_derived',
 'mimiciii_notes']

### Since I connected to `mimic_core`, I can list the tables in this database

In [7]:
conn.list_tables()

['admissions', 'patients', 'transfers']

In [8]:
patients = conn.table("patients")

In [9]:
patients.schema()

ibis.Schema {  
  subject_id         int64
  gender             string
  anchor_age         int64
  anchor_year        int64
  anchor_year_group  string
  dod                date
}

### And do queries

In [10]:
patients.execute(limit=20)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10137012,F,0,2110,2008 - 2010,NaT
1,10156260,M,0,2110,2014 - 2016,NaT
2,10158897,M,0,2110,2008 - 2010,NaT
3,10413283,M,0,2110,2014 - 2016,NaT
4,10660064,M,0,2110,2017 - 2019,NaT
5,10736768,F,0,2110,2017 - 2019,NaT
6,10782862,F,0,2110,2008 - 2010,NaT
7,10942655,M,0,2110,2008 - 2010,NaT
8,10999598,M,0,2110,2008 - 2010,NaT
9,11024137,F,0,2110,2008 - 2010,NaT


In [11]:
conn.table("admissions").execute(limit=20)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,12427812,21593330,2184-01-06 11:51:00,2184-01-10 11:45:00,NaT,URGENT,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,None,UNKNOWN,NaT,NaT,0
1,14029832,22059088,2120-01-18 01:28:00,2120-01-20 16:13:00,NaT,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,None,OTHER,NaT,NaT,0
2,12897089,21776699,2149-02-25 01:40:00,2149-02-27 14:05:00,NaT,URGENT,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,None,ASIAN,NaT,NaT,0
3,16050270,23791681,2161-07-01 21:13:00,2161-07-05 15:45:00,NaT,URGENT,PROCEDURE SITE,HOME,Other,ENGLISH,None,UNKNOWN,NaT,NaT,0
4,14068957,26940015,2130-04-07 10:00:00,2130-04-10 18:44:00,NaT,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,None,UNKNOWN,NaT,NaT,0
5,12501437,22620680,2188-09-08 19:35:00,2188-09-11 14:00:00,NaT,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,None,UNKNOWN,NaT,NaT,0
6,13466375,21011631,2149-08-30 04:30:00,2149-09-29 18:29:00,NaT,URGENT,TRANSFER FROM HOSPITAL,ACUTE HOSPITAL,Medicare,ENGLISH,None,WHITE,NaT,NaT,0
7,10734403,26588273,2110-06-20 14:45:00,2110-06-24 18:15:00,2110-06-24 18:15:00,URGENT,TRANSFER FROM HOSPITAL,None,Other,ENGLISH,None,UNABLE TO OBTAIN,NaT,NaT,1
8,12537266,27845175,2111-09-04 03:52:00,2111-09-16 16:14:00,NaT,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,None,WHITE,NaT,NaT,0
9,10368348,21406513,2160-06-01 03:50:00,2160-06-03 12:00:00,NaT,URGENT,PHYSICIAN REFERRAL,HOME,Medicaid,ENGLISH,None,OTHER,NaT,NaT,0


In [12]:
conn.table("transfers").execute(limit=20)

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,15883097,25063348.0,36143492,ED,Emergency Department,2156-02-18 12:50:00,2156-02-18 19:25:00
1,11911040,23227305.0,37066887,ED,Emergency Department,2181-04-03 11:17:00,2181-04-03 18:03:00
2,15313540,23701153.0,37963493,ED,Emergency Department,2116-02-11 23:18:00,2116-02-12 02:20:00
3,12238614,29387284.0,32928325,ED,Emergency Department,2116-07-31 13:37:00,2116-07-31 22:08:00
4,11222968,25349356.0,37822601,ED,Emergency Department,2176-07-02 10:36:00,2176-07-02 13:49:00
5,17270164,22627031.0,32600471,ED,Emergency Department,2144-10-23 20:58:00,2144-10-24 04:07:00
6,10216527,24501115.0,36542664,ED,Emergency Department,2157-02-05 07:18:00,2157-02-06 13:00:00
7,10645206,20389826.0,31544112,ED,Emergency Department,2151-09-16 14:41:00,2151-09-16 19:20:00
8,19345785,26721665.0,33872320,ED,Emergency Department,2185-09-07 14:37:00,2185-09-07 18:32:00
9,19814293,25844983.0,36185176,ED,Emergency Department,2181-01-16 19:52:00,2181-01-17 03:06:00


### I can connect to another databasae

In [ ]:
conn2 = ibis.bigquery.connect(
    project_id="mimic4-bq",
    dataset_id='physionet-data.mimic_derived',
    credentials=credentials)

In [ ]:
conn2.list_tables()